## 웹 크롤링 

### 인터넷 접속 라이브러리 추가

- urllib.request 모듈로 작업
- 일반 html 소스 로드로 별다른 문제 없음

In [31]:
from urllib.request import urlopen, Request

# 도시별 날씨 검색함수
def get_weather(city):
    # 기상청 홈페이지 도시별 날씨 페이지
    url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page = urlopen(url=url)
    # 인코딩은 utf-8로 되어있다.
    text = page.read().decode('utf-8')
    # 텍스트를 찾는데 꺽쇠 뒤에 city를 찾아라
    text = text[text.find(f'>{city}</a'):]

    # 기온 가져오기
    for i in range(7):
        # 반복하면서 <tb>에 둘러쌓인 원하는 값 찾기
        text = text[text.find(f'<td>')+1:]
    # 찾은 지점부터 3번 째 값
    start = 3
    # 사이값 찾은 뒤에 있는 </tb>값
    end = text.find('</td>')
    # 시작지점 ~ 종료지점 사이 값
    current_temp = text[start:end]
    print(f'{city}의 현재 기온은 {current_temp}℃ 입니다.')

    # 습도 가져오기
    for i in range(3):
        text = text[text.find(f'<td>')+1:]
    start = 3
    end = text.find('</td>')
    current_humid = text[start:end]
    print(f'{city}의 현재 습도는 {current_humid}% 입니다.')
    
if __name__ == '__main__':
    get_weather('부산')

부산의 현재 기온은 11.0℃ 입니다.
부산의 현재 습도는 25% 입니다.


### OpenAPI 크롤링

- 공공데이터 포털
    - https://www.data.go.kr
    - 부산광역시 버스정보안내기 현황 

```PYTHON    
    # SSL 문제 때문에 계속 에러나서 진행이 안됐음
    response = requests.get(total_url, verify=False)
```
- 문제점
    - https를 일반 request로 부르면 SSL 오류발생
    - 해결하려면 외부모듈 request를 사용, (검증)verify=False 옵션을 지정
    - 아래 셀의 39번 라인

In [105]:
import requests # 기본적인 URL 모듈로는 안되서 대체
import json
import ssl
from urllib.parse import quote, unquote, urlencode  # 한글을 URLencode 변환하는 함수

def getRequestUrl(url):
    req = Request(url)

    try:
        res = urlopen(req)
        # 응답이 200으로 나오면 정상적으로 통신된다는 이야기
        if res.getcode() == 200:
            return res.read().decode('utf-8')
    except Exception as e:
        print(e)
        return None

# stationName 정류소 이름
def getDataPortalSearch(stationName, type):
    
    # 고정값인 페이지 주소 + 특수 키
    api_url = 'https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    # 여까지는 일반적인 URL
    queryString = "?" + urlencode(
        {   
            # url을 인코딩해서 특수문자 변환해줌
            'serviceKey' : '6hhxOoRZmduvmq1x2rC8tUpOTEJPythkOXqaCfRhb1G8rL++dNSwoN9DEGcZKHGhumwHaWyhtgGXbNDBbE/J9g==',
            # 페이지 넘버
            '&pageNo' : '1',
            '&numOfRows' : '10',
            # 롯데백화점 이름 자동으로 인코딩
            'stationLoc' : stationName,
            'resultType' : type
            # 시작은 ? 뒤에 임마들은 파라미터(서비스키, &는 구분자, 페이지 넘버 등등)
        }
    )
    total_url = api_url + queryString
    # SSL 문제 때문에 계속 에러나서 진행이 안됐음
    response = requests.get(total_url, verify=False)
    return response.text
    
try:
    result = getDataPortalSearch('학교', 'json')
    # json 읽기
    json_data = json.loads(result)
    # 딕셔너리 형태로 리스트 생성
    station_data = json_data['getTblBusinfoeqStusInfo']['body']['items']['item']
    # 리스트 순번대로 출력
    for item in station_data:
        print(item)
except Exception as e:
    print('찾는 데이터가 없습니다.')

{'stationNum': '16510', 'stationLoc': '동해선기장역.기장중학교', 'lat': '35.24425393', 'lng': '129.2168119', 'addr': '기장군 기장읍 동부리 280-7', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '16511', 'stationLoc': '동해선기장역.기장중학교', 'lat': '35.24384863', 'lng': '129.2165536', 'addr': '기장군 기장읍 동부리 162-20', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '16546', 'stationLoc': '교리초등학교', 'lat': '35.25427637', 'lng': '129.2163343', 'addr': '기장군 기장읍 교리 221', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '16549', 'stationLoc': '기장초등학교', 'lat': '35.24387104', 'lng': '129.2134572', 'addr': '기장군 기장읍 동부리 175', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '16555', 'stationLoc': '교리초등학교', 'lat': '35.25416625', 'lng': '129.2154621', 'addr': '기장군 기장읍 323-7', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '16563', 'stationLoc': '교리초등학교', 'lat': '35.25398191', 'lng': '129.2155446', 'addr': '기장읍 차성로 414', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '16271', 'stationLoc'

### OpenAPI로 가져온 데이터 지도 표시

```python
stop_str = '<h4>' + item['stationLoc'] + '</h4>' + item['stationNum'] + '<br>' + item['addr']
iframe = folium.IFrame(stop_str)
```

- 문제점
    - 딕셔너리 구조하고 문자열 포맷팅 f'{}' 호환 안된다.
    - 구식방법인 문자열 결합 방식으로 해결 

In [106]:
import folium

if len(station_data) > 0:       
    # 정류소 중 제일 첫번째 인덱스 정류소 위경도를 중심으로
    center_lat = station_data[0]['lat']
    center_lng = station_data[0]['lng']
    
m = folium.Map(location=[center_lat, center_lng],
               zoom_start=13)

# 전체 정류소 위치값 마커 표시
for item in station_data:
        stop_str = '<h4>' + item['stationLoc'] + '</h4>' + item['stationNum'] + '<br>' + item['addr']
        iframe = folium.IFrame(stop_str)
        folium.Marker(location=[item['lat'],
                                item['lng']],
                                popup=folium.Popup(iframe,
                                                   min_width=200, 
                                                   max_width=250),
                                icon=folium.Icon(color='blue', 
                                                 icon='pushpin')
                      ).add_to(m)
m

In [96]:
import folium

if len(station_data) > 0:       
    # 정류소 중 제일 첫번째 인덱스 정류소 위경도를 중심으로
    center_lat = station_data[0]['lat']
    center_lng = station_data[0]['lng']
    
m = folium.Map(location=[center_lat, center_lng],
               zoom_start=13)

# 전체 정류소 위치값 마커 표시
for item in station_data:
        folium.Marker(location=[item['lat'],
                                item['lng']],
                                popup=folium.Popup(folium.IFrame('<h4>' + item['stationLoc'] + 
                                                                 '</h4>' + item['stationNum'] + 
                                                                 '<br>' + item['addr']),
                                                   min_width=200, 
                                                   max_width=250),
                                icon=folium.Icon(color='blue', 
                                                 icon='pushpin')
                     ).add_to(m)
m

### BeautifulSoup (version 4)

웹크롤링을 편하게 해주는 도구(라이브러리)

```shell
pip install beautifulsoup4
```

In [108]:
!pip install beautifulsoup4

     ---------------------------------------- 0.0/129.4 kB ? eta -:--:--
     -------------------------------------- 129.4/129.4 kB 3.8 MB/s eta 0:00:00


#### 모듈 import

```python
from bs4 import BeaurifulSoup
```

In [109]:
from bs4 import BeaurifulSoup
import requests


ImportError: cannot import name 'BeaurifulSoup' from 'bs4' (c:\DEV\Langs\Python3.11.1\Lib\site-packages\bs4\__init__.py)